In [1]:
import openmeteo_requests
import matplotlib as plt
import requests_cache
import pandas as pd
from retry_requests import retry
# from powderalert.ml_logic.preprocessor import define_X, preprocess
# from powderalert.ml_logic.data import clean_data
from datetime import datetime as dt
from datetime import timedelta

In [2]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 47.26580883196723,
	"longitude": 11.84457426992035,
    'past_days': 7,
    'forecast_days': 0,
	"hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "precipitation", "rain", "snowfall", "snow_depth", "weather_code", "pressure_msl", "surface_pressure", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "et0_fao_evapotranspiration", "vapour_pressure_deficit", "wind_speed_10m", "wind_speed_120m", "wind_direction_10m", "wind_direction_120m", "wind_gusts_10m", "soil_temperature_0cm", "soil_temperature_6cm", "soil_temperature_18cm", "soil_temperature_54cm", "soil_moisture_0_to_1cm", "soil_moisture_1_to_3cm", "soil_moisture_3_to_9cm", "soil_moisture_9_to_27cm", "soil_moisture_27_to_81cm"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")


Coordinates 47.2400016784668°N 11.84000015258789°E
Elevation 1818.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [4]:
# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(3).ValuesAsNumpy()
hourly_rain = hourly.Variables(4).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(5).ValuesAsNumpy()
hourly_snow_depth = hourly.Variables(6).ValuesAsNumpy()
hourly_weather_code = hourly.Variables(7).ValuesAsNumpy()
hourly_pressure_msl = hourly.Variables(8).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(9).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(10).ValuesAsNumpy()
hourly_cloud_cover_low = hourly.Variables(11).ValuesAsNumpy()
hourly_cloud_cover_mid = hourly.Variables(12).ValuesAsNumpy()
hourly_cloud_cover_high = hourly.Variables(13).ValuesAsNumpy()
hourly_et0_fao_evapotranspiration = hourly.Variables(14).ValuesAsNumpy()
hourly_vapour_pressure_deficit = hourly.Variables(15).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(16).ValuesAsNumpy()
hourly_wind_speed_120m = hourly.Variables(17).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(18).ValuesAsNumpy()
hourly_wind_direction_120m = hourly.Variables(19).ValuesAsNumpy()
hourly_wind_gusts_10m = hourly.Variables(20).ValuesAsNumpy()
hourly_sunshine_duration = hourly.Variables(21).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["dew_point_2m"] = hourly_dew_point_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["snowfall"] = hourly_snowfall
hourly_data["snow_depth"] = hourly_snow_depth
hourly_data["weather_code"] = hourly_weather_code
hourly_data["pressure_msl"] = hourly_pressure_msl
hourly_data["surface_pressure"] = hourly_surface_pressure
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
hourly_data["cloud_cover_high"] = hourly_cloud_cover_high
hourly_data["et0_fao_evapotranspiration"] = hourly_et0_fao_evapotranspiration
hourly_data["vapour_pressure_deficit"] = hourly_vapour_pressure_deficit
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_speed_100m"] = hourly_wind_speed_120m #changing column names to match train dataset
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
hourly_data["wind_direction_100m"] = hourly_wind_direction_120m #changing column names to match train dataset
hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
hourly_data["sunshine_duration"] = hourly_sunshine_duration

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

                         date  temperature_2m  relative_humidity_2m  \
0   2024-12-10 00:00:00+00:00          -6.774                 100.0   
1   2024-12-10 01:00:00+00:00          -6.874                 100.0   
2   2024-12-10 02:00:00+00:00          -6.824                 100.0   
3   2024-12-10 03:00:00+00:00          -6.724                 100.0   
4   2024-12-10 04:00:00+00:00          -6.924                 100.0   
..                        ...             ...                   ...   
163 2024-12-16 19:00:00+00:00           4.026                  53.0   
164 2024-12-16 20:00:00+00:00           3.926                  52.0   
165 2024-12-16 21:00:00+00:00           4.376                  50.0   
166 2024-12-16 22:00:00+00:00           4.376                  51.0   
167 2024-12-16 23:00:00+00:00           4.026                  54.0   

     dew_point_2m  precipitation  rain  snowfall  snow_depth  weather_code  \
0       -6.774000            0.0   0.0       0.0        0.24         

In [5]:
hourly_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   date                        49 non-null     datetime64[ns, UTC]
 1   temperature_2m              49 non-null     float32            
 2   relative_humidity_2m        49 non-null     float32            
 3   dew_point_2m                49 non-null     float32            
 4   precipitation               49 non-null     float32            
 5   rain                        49 non-null     float32            
 6   snowfall                    49 non-null     float32            
 7   snow_depth                  49 non-null     float32            
 8   weather_code                49 non-null     float32            
 9   pressure_msl                49 non-null     float32            
 10  surface_pressure            49 non-null     float32            


In [6]:
hourly_dataframe.date

0    2024-12-17 11:00:00+00:00
1    2024-12-17 12:00:00+00:00
2    2024-12-17 13:00:00+00:00
3    2024-12-17 14:00:00+00:00
4    2024-12-17 15:00:00+00:00
5    2024-12-17 16:00:00+00:00
6    2024-12-17 17:00:00+00:00
7    2024-12-17 18:00:00+00:00
8    2024-12-17 19:00:00+00:00
9    2024-12-17 20:00:00+00:00
10   2024-12-17 21:00:00+00:00
11   2024-12-17 22:00:00+00:00
12   2024-12-17 23:00:00+00:00
13   2024-12-18 00:00:00+00:00
14   2024-12-18 01:00:00+00:00
15   2024-12-18 02:00:00+00:00
16   2024-12-18 03:00:00+00:00
17   2024-12-18 04:00:00+00:00
18   2024-12-18 05:00:00+00:00
19   2024-12-18 06:00:00+00:00
20   2024-12-18 07:00:00+00:00
21   2024-12-18 08:00:00+00:00
22   2024-12-18 09:00:00+00:00
23   2024-12-18 10:00:00+00:00
24   2024-12-18 11:00:00+00:00
25   2024-12-18 12:00:00+00:00
26   2024-12-18 13:00:00+00:00
27   2024-12-18 14:00:00+00:00
28   2024-12-18 15:00:00+00:00
29   2024-12-18 16:00:00+00:00
30   2024-12-18 17:00:00+00:00
31   2024-12-18 18:00:00+00:00
32   202

In [5]:
api_columns = ['date', 'temperature_2m', 'relative_humidity_2m', 'dew_point_2m',
       'precipitation', 'rain', 'snowfall', 'snow_depth', 'weather_code',
       'pressure_msl', 'surface_pressure', 'cloud_cover', 'cloud_cover_low',
       'cloud_cover_mid', 'cloud_cover_high', 'et0_fao_evapotranspiration',
       'vapour_pressure_deficit', 'wind_speed_10m', 'wind_speed_100m',
       'wind_direction_10m', 'wind_direction_100m', 'wind_gusts_10m',
       'sunshine_duration']